# NLE Assessed Coursework 3: Question 1

For this assessment, you are expected to complete and submit 4 notebook files.  There is 1 notebook file for each question (to speed up load times).  This is notebook 1 out of 4.

Marking guidelines are provided as a separate document.

In order to provide unique datasets for analysis by different students, you must enter your candidate number in the following cell.

In [1]:
candidateno=198735 #this MUST be updated to your candidate number so that you get a unique data sample


In [2]:
#preliminary imports
import sys
sys.path.append(r'/Users/Joe/Documents/Python Scripts/resources/resources')
sys.path.append(r'\\ad.susx.ac.uk\ITS\TeachingResources\Departments\Informatics\LanguageEngineering\resources')

import re
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from itertools import zip_longest
from nltk.tokenize import word_tokenize

from sussex_nltk.corpus_readers import AmazonReviewCorpusReader
import random
from nltk.corpus import stopwords
import spacy
#nlp=spacy.load('en')
nlp=spacy.load('en_core_web_sm')
from nltk.corpus import gutenberg

Sussex NLTK root directory is /Users/Joe/Documents/Python Scripts/resources/resources


## Question 1: Part-of-Speech Tagging (25 marks)


In [3]:
#Do NOT change the code in this cell.

#preparing corpus
def display_sent(asent):
    headings=["token","lower","lemma","pos","NER"]
    info=[]
    for t in asent:
        info.append([t.text,t.lower_,t.lemma_,t.pos_,t.ent_type_])
    return(pd.DataFrame(info,columns=headings))

def tag_sent(asent):
    tagged=[]
    for t in asent:
        tagged.append((t.lower_,t.pos_))
    return tagged

def clean_text(astring):
    #replace newlines with space
    newstring=re.sub("\n"," ",astring)
    #remove title and chapter headings
    newstring=re.sub("\[[^\]]*\]"," ",newstring)
    newstring=re.sub("VOLUME \S+"," ",newstring)
    newstring=re.sub("CHAPTER \S+"," ",newstring)
    newstring=re.sub("\s\s+"," ",newstring)
    #return re.sub("([^\.|^ ])  +",r"\1 .  ",newstring).lstrip().rstrip()
    return newstring.lstrip().rstrip()

def display_tags(sentslist):
    taglist={}
    for s in sentslist:
        for t in s:
            form=t.lower_
            pos=t.pos_
            taglist[pos]=taglist.get(pos,0)+1
    print(len(taglist.keys()))
    print(taglist)
        
def get_train_test(sentslist,seed=candidateno):
    random.seed(seed)
    random.shuffle(sentslist)
    testsize=10
    train=[tag_sent(s) for s in sentslist[testsize:]]
    test=[tag_sent(s) for s in sentslist[:testsize]]
    return train,test
    
alice=clean_text(gutenberg.raw('carroll-alice.txt'))
nlp_alice=list(nlp(alice).sents)

The code below will generate (unique to you) training and test sets of pos-tagged sentences from the novel "Alice in Wonderland"

In [4]:
#do not change the code in this cell
allsents=list(nlp_alice)
train,test=get_train_test(allsents)

In the cell below, a class `unigram_tagger` is defined, which will be used in parts a and b of this question

In [5]:
#do not change the code in this cell
class unigram_tagger():
    
    def __init__(self,trainingdata=[]):
        self.tagcounts={}
        self.wordcounts={}
        self.tagperwordcounts={}
        self.train(trainingdata=trainingdata)
        
    def train(self,trainingdata):
        
        for sentence in trainingdata:
            for token,tag in sentence:
                self.tagcounts[tag]=self.tagcounts.get(tag,0)+1
                self.wordcounts[token]=self.wordcounts.get(token,0)+1
                current=self.tagperwordcounts.get(token,{})
                current[tag]=current.get(tag,0)+1
                self.tagperwordcounts[token]=current
                

a) Create an instance of the `unigram_tagger` class which is trained on your training sentences from "Alice in Wonderland" (stored in `train`).
Explain what is stored, after training, in the three instance variables:
* `self.tagcounts`
* `self.wordcounts`
* `self.tagperwordcounts`

You should refer to the code and specific examples from the training data. \[4 marks \]

In [6]:
tag_instance = unigram_tagger(train)


**self.tagcounts** 

*tagcounts* stores a dictionary in which the keys are the type of speech tag, and the values are the number of times they have been assigned in the particular instance.
For example: the tag *'PRON'* has been assigned to 3219 words in our training data.

**self.wordcounts**

*wordcounts* stores a dictionary in which the keys are words in the training data and the values are the number of times they have been seen in the particular instance.
For example: the word *'i'* has been seen 524 times in our training data.

**self.tagperwordcounts**

*tagperwordcounts* stores a dictionary in which the keys are words in the training data and the values are dictionaries for tag counts of that word.
For example: the word *'have'* has been seen a total of 87 times. 83 times its been tagged as *'AUX'* and 4 times it was tagged as *'VERB'*. And is shown like so: `{'AUX': 83, 'VERB': 4}`.

b) In the cells below, **extend** the code for the `unigram_tagger` class so that it also has a `tag()` method.  This method should assign the tag, $t$, which maximises the unigram tag probability, $P(t|w)$, for the observed word, $w$.  **Evaluate** the performance of the `unigram_tagger` on your test sentences.  **Discuss** your results. \[8 marks\] 



In [7]:
class unigram_tagger():
    
    def __init__(self,trainingdata=[]):
        self.tagcounts={}
        self.wordcounts={}
        self.tagperwordcounts={}
        self.train(trainingdata=trainingdata)
        
    def train(self,trainingdata):
        
        for sentence in trainingdata:
            for token,tag in sentence:
                self.tagcounts[tag]=self.tagcounts.get(tag,0)+1
                self.wordcounts[token]=self.wordcounts.get(token,0)+1
                current=self.tagperwordcounts.get(token,{})
                current[tag]=current.get(tag,0)+1
                self.tagperwordcounts[token]=current
                
    def tag(self, word):  
        tag_set = self.tagperwordcounts.get(word,0)
        if (tag_set != 0):
            tags = tag_set.keys()
            total = sum(tag_set.values())
            tagged = ''
            val = 0
            for key in tags:
                current = tag_set[key]/total
                if val < current:
                    tagged = key
            return tagged

In [8]:
tag_instance = unigram_tagger(train)
correct = 0
total = 0
for sentence in test:
    for word,tagged in sentence:
        assigned_tag = tag_instance.tag(word)
        #print(word,tagged, assigned_tag)
        if (assigned_tag == tagged):
            correct += 1
        total += 1    
accuracy = correct/total * 100
print("{} tagged correctly out of a total of {} words".format(correct,total))
print("This tagger has {} accuracy".format(accuracy))

99 tagged correctly out of a total of 165 words
This tagger has 60.0 accuracy


#### Results
A word's context is necessary in order to correctly identify what tag to give it i.e. the word *'said'* can be a verb or a pronoun. The tagger it chose to tag it as a verb instead of a pronoun, due to the limited times that the word *'said'* is used as a pronoun. As it is more likely to be a verb the tagger tags it as a verb. It doesn't look at any context. The tagger did not have the expected 90% accuracy bu returned a 60% accuracy.

The above could be a result of the following:
1. Not enough test data, to test the tagger on. This probably isn't the case - this type of tagger's training data above a certain threshold will only increase in proportion i.e. Here just one sentence in the training data had *'said'* being used in a pronoun context, scale the number of training data up; this number would increase in proportion to the verb cases. 
2. The amount of test data actually used, the more the tagger is tested accuracy may increase. The test sentences where the word is a pronoun are just anomalies in the data. Therefore, an increase in test data could show better potential for this tagger.

In [9]:
# 'said' shown here for context
tag_instance.tagperwordcounts['said']

{'VERB': 453, 'PROPN': 1}

Overall, this tagger is not very accurate for a reason, and even an increase in test data may not even increase its accuracy, because at the end of the day the tagger is taking an educated guess. This is the most likely tag for this word. This isn't a very effective way of tagging sentences, and therefore can be improved upon.

In the cell below, a `hmm_tagger` class is defined, which will be used in parts c and d of this question.

In [10]:
#do not change the code in this cell
class hmm_tagger():
    
    def __init__(self,trainingdata=[]):
        
        self.emissions={}
        self.transitions={}
        self.train(trainingdata=trainingdata)
        
    def train(self,trainingdata):
        
        for sentence in trainingdata:
            previous="START"
            for token,tag in sentence:
                
                current=self.emissions.get(tag,{})
                current[token]=current.get(token,0)+1
                self.emissions[tag]=current
                bigram=self.transitions.get(previous,{})
                bigram[tag]=bigram.get(tag,0)+1
                self.transitions[previous]=bigram
                previous=tag
                
        self.emissions={tag:{token:freq/sum(countdict.values()) for (token,freq) in countdict.items()}for (tag,countdict) in self.emissions.items()}
        self.transitions={tag:{token:freq/sum(countdict.values()) for (token,freq) in countdict.items()}for (tag,countdict) in self.transitions.items()}

c) **Create** an instance of the `hmm_tagger` class which is trained on your training sentences from "Alice in Wonderland" (stored in `train`).  With reference to your testing sentences (stored in `test`), **explain**
* how to calculate the probability of an observed sequence of words for a given sequence of tags: $$P(w_1^n|t_1^n)$$
* how to calculate the probablity of a possible sequence of tags for a given sequence of words: $$P(t_1^n|w_1^n)$$

\[6 marks\]

In [11]:
hmm_instance = hmm_tagger(train)

#### Forward Algorithm

To calculate the probability of an observed sequence of words for a given sequence of tags you use the forward algorithm: $$P(w_1^n|t_1^n) = \prod_{i}^n P(w_i|t_i)$$
This is the product of the emissions in the word sequence, relating to the word and its tag in the sequence. These probabilities are calculated when the `hmm_tagger` is trained, and thus can be used to calculate the probability above. For example with the seventh test sentence (due to its size it is easy to describe):

In [12]:
print(test[7])

[('i', 'PRON'), ('suppose', 'VERB'), ('so', 'ADV'), (',', 'PUNCT'), ("'", 'PUNCT'), ('said', 'VERB'), ('alice', 'PROPN'), ('.', 'PUNCT'), ("'", 'PUNCT')]


Therefore for this sentence the probability is $P(i suppose so , ' said alice . ' |PRON VERB ADV PUNCT PUNCT VERB PROPN PUNCT PUNCT)$ which refers to the probability of each word being this tag multiplied together. To calculate this we need to know these probabilities, i.e. $P(i|PRON)$ as seen below. The product of these probabilities gives us the probability of an observed sequence of words for a given sequence of tags, also shown below in the example:

In [13]:
print('P(i|PRON) = {}'.format(hmm_instance.emissions['PRON']['i']))
print('P(suppose|VERB) = {}'.format(hmm_instance.emissions['VERB']['suppose']))
print('P(so|ADV) = {}'.format(hmm_instance.emissions['ADV']['so']))
print('P(,|PUNCT) = {}'.format(hmm_instance.emissions['PUNCT'][',']))
print("P('|PUNCT) = {}".format(hmm_instance.emissions['PUNCT']["'"]))
print('P(said|VERB) = {}'.format(hmm_instance.emissions['VERB']['said']))
print('P(alice|PROPN) = {}'.format(hmm_instance.emissions['PROPN']['alice']))
print('P(.|PUNCT) = {}'.format(hmm_instance.emissions['PUNCT']['.']))
print("P('|PUNCT) = {}".format(hmm_instance.emissions['PUNCT']["'"]))

probability = (hmm_instance.emissions['PRON']['i'] * 
               hmm_instance.emissions['VERB']['suppose'] * 
               hmm_instance.emissions['ADV']['so'] * 
               hmm_instance.emissions['PUNCT'][','] * 
               hmm_instance.emissions['PUNCT']["'"] * 
               hmm_instance.emissions['VERB']['said'] * 
               hmm_instance.emissions['PROPN']['alice'] * 
               hmm_instance.emissions['PUNCT']['.'] * 
               hmm_instance.emissions['PUNCT']["'"] )
print("P(i suppose so , ' said alice . ' |PRON VERB ADV PUNCT PUNCT VERB PROPN PUNCT PUNCT) = {}".format(probability))

P(i|PRON) = 0.16278347312830072
P(suppose|VERB) = 0.002607561929595828
P(so|ADV) = 0.0610079575596817
P(,|PUNCT) = 0.3343112601285275
P('|PUNCT) = 0.3065101983794356
P(said|VERB) = 0.0984354628422425
P(alice|PROPN) = 0.26375838926174494
P(.|PUNCT) = 0.13467449008102822
P('|PUNCT) = 0.3065101983794356
P(i suppose so , ' said alice . ' |PRON VERB ADV PUNCT PUNCT VERB PROPN PUNCT PUNCT) = 2.843907720535083e-09


To conclude, this algorithm finds the probabilities for each emission, and times them together to give the proability of this word sequence given this tag sequence. 

#### Decoding Algorithm
To calculate the probability of a possible sequence of tags for a given sequence of words you use the decoding algorithm: $$P(t_1^n|w_1^n) \approx \prod_{i}^n P(w_i|t_i) P(t_i|t_{i-1})$$
This is the product of the emissions for that word and tag multiplied by the the transitions for that tag sequence. Therefore for the seventh sentence we take the probability of this word sequence given the tag sequence we are testing and multiply this by the probability of a particular tag sequence, and this is the probability of a possible sequence of tags for a given sequence of words. In this case it would be $P((i suppose so , ' said alice . ' |PRON VERB ADV PUNCT PUNCT VERB PROPN PUNCT PUNCT)$ multiply the probability of the tag sequence `PRON VERB ADV PUNCT PUNCT VERB PROPN PUNCT PUNCT`. This is the tag sequence used in the last calculation to show how to calculate this. But there are $k^n$ possible tag sequences where k is the tagset and n is the amount of words in the sequence; therefore in this eaxample there is a total of $15^7$ which equals $170,859,375$ different tag sequences. Given a computer's standards this is not an efficient way to find the most likely tag sequence. However, this is irrelevant as the algorithm just needs to calculate the proability of just one of these tag sequences. The calculations for the example are shown below:

In [14]:
print('The probabilities needed are:')
print('The proability of a Pron being the starting word P(PRON|START): {}'.format(hmm_instance.transitions['START']['PRON']))
print('The proability of a Verb being after a Pron P(VERB|PRON): {}'.format(hmm_instance.transitions['PRON']['VERB']))
print('The proability of a Adv being after a Verb P(ADV|VERB): {}'.format(hmm_instance.transitions['VERB']['ADV']))
print('The proability of a Punct being after a Adv P(PUNCT|ADV): {}'.format(hmm_instance.transitions['ADV']['PUNCT']))
print('The proability of a Punct being after a Punct P(PUNCT|PUNCT): {}'.format(hmm_instance.transitions['PUNCT']['PUNCT']))
print('The proability of a Verb being after a Punct P(VERB|PUNCT): {}'.format(hmm_instance.transitions['PUNCT']['VERB']))
print('The proability of a Propn being after a Verb P(PROPN|VERB): {}'.format(hmm_instance.transitions['VERB']['PROPN']))
print('The proability of a Punct being after a Propn P(PUNCT|PROPN): {}'.format(hmm_instance.transitions['PROPN']['PUNCT']))
print('The proability of a Punct being after a Punct P(PUNCT|PUNCT): {}'.format(hmm_instance.transitions['PUNCT']['PUNCT']))

probability2= (probability *
(hmm_instance.transitions['START']['PRON'] *
hmm_instance.transitions['PRON']['VERB'] *
hmm_instance.transitions['VERB']['ADV'] *
hmm_instance.transitions['ADV']['PUNCT'] *
hmm_instance.transitions['PUNCT']['PUNCT'] *
hmm_instance.transitions['PUNCT']['VERB'] *
hmm_instance.transitions['VERB']['PROPN'] *
hmm_instance.transitions['PROPN']['PUNCT'] *
hmm_instance.transitions['PUNCT']['PUNCT']))
print("The probability P(PRON VERB ADV PUNCT PUNCT VERB PROPN PUNCT PUNCT| i suppose so , ' said alice . ' ) is: {}".format(probability2))

The probabilities needed are:
The proability of a Pron being the starting word P(PRON|START): 0.23701462430660616
The proability of a Verb being after a Pron P(VERB|PRON): 0.42159303049159924
The proability of a Adv being after a Verb P(ADV|VERB): 0.08714596949891068
The proability of a Punct being after a Adv P(PUNCT|ADV): 0.2832369942196532
The proability of a Punct being after a Punct P(PUNCT|PUNCT): 0.387797506611258
The proability of a Verb being after a Punct P(VERB|PUNCT): 0.10256894597657726
The proability of a Propn being after a Verb P(PROPN|VERB): 0.03965141612200436
The proability of a Punct being after a Propn P(PUNCT|PROPN): 0.45143638850889195
The proability of a Punct being after a Punct P(PUNCT|PUNCT): 0.387797506611258
The probability P(PRON VERB ADV PUNCT PUNCT VERB PROPN PUNCT PUNCT| i suppose so , ' said alice . ' ) is: 1.936700801345751e-15


Now shown above is the probability that given the seventh test sentence that its tag sequence is `PRON VERB ADV PUNCT PUNCT VERB PROPN PUNCT PUNCT`. This is done by taking each probability for that tag occurring after the previous tag i.e for example $P(ADV|VERB)$ multiplied together and multiplied by the probability of that word sequence given the tag sequence. 

d) Using one of your `test` sentences as an example, **explain** how the Viterbi algorithm can be used to find the most likely tag sequence.  You do not need to write code for this question but should explain the calculations that need to be made at each step.  **Comment** on whether the sequence found by the Viterbi algorithm is correct for your chosen test sentence.  **Discuss** why using the Viterbi algorithm is better than the brute-force approach of enumerating and testing all tag sequence possibilities.\[7 marks\]


The Viterbi algorithm can find the most likely sequence by using the power of dynamic programming. It splits the sentences into each word, and analyses the most likely tag for each word in order to correctly identify the best tag sequence for that word seqeunce. It uses HMM assumptions in order to do this; but unlike HMM its approach eliminates the need to explore all possibilities. It uses the HMM assumptions to work out based on the previous word potentially being tag x whether the current word is more likely to be tag x or tag y etc. It then checks again using tag y, for each tag, the previous word could be. This is slightly different for the base case, but after this the algorithm is recursive for every step, and returns the highest probable tag sequence once it has recursed through the entire word sequence. Now before starting, the algorithm numbers each word in the sequence from 1 to n where n is the length of the word sequence. In our train and testing corpus, there are 7 different tags that a word could have. Therefore for the base case, each of these tag probabilities is considered for the first recursive step, as the recursive step requires previous word tag probabilities. A calculation is performed by splitting each possible classification into subproblems.

So for the base case take our test sentence as an example:

In [15]:
print(test[7])

[('i', 'PRON'), ('suppose', 'VERB'), ('so', 'ADV'), (',', 'PUNCT'), ("'", 'PUNCT'), ('said', 'VERB'), ('alice', 'PROPN'), ('.', 'PUNCT'), ("'", 'PUNCT')]


The first subproblem of the base case might be if *'i'* is the PRON tag. Therefore this probability $V(1,PRON)$ where V stands for Viterbi, 1 being the first word in the sequence and PRON is the tag being considered. 
To calculate, take the probability that this word given the tag PRON, i.e. $P(i|PRON)$, multiplied by the probability of the tag being at the start of the sentence i.e $P(PRON|start)$. So $V(1,PRON) = P(i|PRON) \times P(PRON|start)$ This is repeated for each of the tags that this word could be, before moving on.

After the initial base case, is the algorithm's recursive case. Every case after the first is the same with the numbers changed. The recursive step is as follows: $$V(i,t) = \max_{t'∈ T}(V(i-1,t') \times P(t|t') \times P(w_i|t))$$ To calculate the probability of the 2nd word being a certain tag in our example, take the probability that the previous word was the tag *'PRON'* for example, represented as $V(1,PRON)$ in this case, and multiply it by the probability that this word is a certain tag given the previous tag, for example if the tag being tested is *'PRON'* then the probability would be $P(PRON|PRON)$. This is then multiplied by the probability of this word given in the testing tag, in this case it would be $P(suppose|PRON)$. The probabilities for every other possible tag is calculated i.e. the 6 others, and choose the highest probability for our chosen tag i.e. $V(2,PRON)$. This subproblem is then repeated 6 other times for every other tag that the 2nd word could be, resulting in 7 probabilities each with their 2 possible tag sequences. Repeat this whole process until the end of the word sequence then choose the highest probable tag sequence. 

In [16]:
for word,tag in test[7]:
    print("{} {}".format(word,tag_instance.tagperwordcounts[word]))

i {'PRON': 524}
suppose {'VERB': 12}
so {'ADV': 138, 'CCONJ': 3, 'SCONJ': 7}
, {'PUNCT': 2393}
' {'PUNCT': 2194, 'PART': 6}
said {'VERB': 453, 'PROPN': 1}
alice {'PROPN': 393}
. {'PUNCT': 964}
' {'PUNCT': 2194, 'PART': 6}


Shown above are the tag counts for each word in the sentence. Every other tag in our tagset, that isn't these counts, is therefore 0 and the probability of that tag for that word is therefore 0. For example the first word's tag can only be *'PRON'* and the second word can only be *'VERB'*. Bear in mind that the algorithm does every tag in the tagset but all of these tags expect the ones mentioned would be 0.

Moving on to the third word there are 3 possible tags, the rest are therefore 0 when considered. This results in 3 sub problems. With each do $V(2,VERB) \times P(tag x|VERB) \times P(so|tag x)$ *'tagx'* is replaced with each of the tags in the tagset. This results in *'ADV'* being chosen. This is repeated for every other word in the sentence, arriving at the tag sequence: `PRON VERB ADV PUNCT PUNCT VERB PROPN PUNCT PUNCT` which is the same as the given tags and therefore correct.

The efficiency of Viterbi is better than the brute force method of HMM. After a certain point the Viterbi algorithm is quicker due to the smaller number of comparisons and problems it needs to do. The efficiency of HMM is $k^n$ whereas the efficiency of Viterbi is $k^2 \times n$ which is smaller than HMM complexity. This means that after a threshold the Viterbi algorithm is more efficient and does less sub-problems than HMM.

In [2]:
##This code will word count all of the markdown cells in the notebook saved at filepath
##Running it before providing any answers shows that the questions have a word count of 1202

import io
from nbformat import current

filepath="a3_1.ipynb"
question_count=533

with io.open(filepath, 'r', encoding='utf-8') as f:
    nb = current.read(f, 'json')

word_count = 0
for cell in nb.worksheets[0].cells:
    if cell.cell_type == "markdown":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))
print("Submission length is {}".format(word_count-question_count))

Submission length is 1484
